In [22]:
from langchain.chat_models import init_chat_model
from langchain.agents import initialize_agent, AgentType
from langchain.tools import Tool
from langchain.memory import ConversationBufferWindowMemory
from langchain_community.tools.tavily_search import TavilySearchResults
from langchain.chat_models import init_chat_model
from langchain.memory import ConversationBufferMemory
from langchain.schema import HumanMessage, AIMessage

In [5]:
import getpass
import os

os.environ["LANGSMITH_TRACING"] = "true"
os.environ["LANGSMITH_API_KEY"] = getpass.getpass("LANGSMITH_API_KEY: \n")

os.environ["TAVILY_API_KEY"] = getpass.getpass("TAVILY_API_KEY: \n")

In [23]:
# Simple model setup
model = init_chat_model(
    "llama3",
    model_provider="ollama",
    model_kwargs={"max_tokens": 150, "temperature": 0}
)

In [24]:
# Simple memory
memory = ConversationBufferMemory(return_messages=True)


In [25]:
class SimpleAgent:
    def __init__(self, model, memory):
        self.model = model
        self.memory = memory
    
    def chat(self, message):
        # Get conversation history
        history = self.memory.chat_memory.messages
        
        # Create context with history
        context = "You are a helpful assistant. Remember information from our conversation.\n\n"
        
        # Add conversation history
        for msg in history[-6:]:  # Last 6 messages only
            if isinstance(msg, HumanMessage):
                context += f"Human: {msg.content}\n"
            elif isinstance(msg, AIMessage):
                context += f"Assistant: {msg.content}\n"
        
        # Add current message
        context += f"Human: {message}\nAssistant:"
        
        try:
            # Get response
            response = self.model.invoke(context)
            response_text = response.content
            
            # Save to memory
            self.memory.chat_memory.add_user_message(message)
            self.memory.chat_memory.add_ai_message(response_text)
            
            return response_text
            
        except Exception as e:
            return f"Error: {str(e)}"
    
    def show_memory(self):
        messages = self.memory.chat_memory.messages
        print(f"Memory has {len(messages)} messages:")
        for i, msg in enumerate(messages):
            msg_type = "Human" if isinstance(msg, HumanMessage) else "AI"
            print(f"{i}: {msg_type}: {msg.content}")

In [26]:
# Create and test the simple agent
print("=== Creating Simple Agent ===")
simple_agent = SimpleAgent(model, memory)

# Test 1: Greeting with name
print("\n1. Greeting with name...")
response1 = simple_agent.chat("Hello! My name is Nerea. Please remember this for our conversation.")
print(f"Agent: {response1}")

=== Creating Simple Agent ===

1. Greeting with name...
Agent: Nice to meet you, Nerea! I've taken note that your name is Nerea. I'll make sure to remember it throughout our conversation. What would you like to talk about or ask?


In [27]:
print("\n2. Testing memory...")
response2 = simple_agent.chat("What is my name?")
print(f"Agent: {response2}")


2. Testing memory...
Agent: Your name is Nerea! We discussed this at the beginning of our conversation, remember?


In [28]:
# Show memory contents
print("\n=== Memory Contents ===")
simple_agent.show_memory()


=== Memory Contents ===
Memory has 4 messages:
0: Human: Hello! My name is Nerea. Please remember this for our conversation.
1: AI: Nice to meet you, Nerea! I've taken note that your name is Nerea. I'll make sure to remember it throughout our conversation. What would you like to talk about or ask?
2: Human: What is my name?
3: AI: Your name is Nerea! We discussed this at the beginning of our conversation, remember?
